# Методы сбора и обработки данных из Интернет

## Урок 2. Практическое задание

### Задание 1

Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы) с сайта superjob.ru и hh.ru. Приложение должно анализировать несколько страниц сайта (также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:

    *Наименование вакансии
    *Предлагаемую зарплату (отдельно мин. и отдельно макс. и отдельно валюта)
    *Ссылку на саму вакансию        
    *Сайт откуда собрана вакансия
По своему желанию можно добавить еще работодателя и расположение. Данная структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas.

**Примечания:**

В домашнем задании о hh.ru ссылку на следующую страницу нужно будет получать из узла. Обработали страницу, получили ссылку на следующую из узла, сделали GET-запрос - у вас страницы и вы ее снова обрабатываете.

Чтобы получить только вакансии, без рекламы - делем поиск по class.
Сначала обрабатываем первую страницу.
Затем вторую и следующие страницы (&page=1 и т.д.). Но так можно попасть на страницу без требуемой информации.

На символе последней страницы код следующий:
<a class="bloko-button HH-Pager-Control" data-page="6" data-qa="pager-page" 
   
Более удобный способ - кнопка "Дальше". Когда дойдем до последней страницы, то кнопки "Дальше" не будет.
Нужно будет проверять на каждой странице наличие кнопки "Дальше". Если она есть, извлекаем атрибут href и идем дальше. Если кнопки "Дальше" нет - все закончено, складываете в общую структуру, выдаете результат.

На superjob.ru такой же подход, только первая страница с цифрой 1, а не 0, как на hh.ru.
На superjob.ru названия class нечеловекочитаемые.

Если даныне о зарплате отсутствуют, то пишите None. Значения зарплаты должны быть типа int.

In [1]:
from bs4 import BeautifulSoup as bs
import requests
import re
import pandas as pd
from pprint import pprint

In [2]:
def _parser_hh(vacancy):

    vacancy_data = []
    
    params = {
        'text': vacancy, \
        'L_save_area':True, \
        'clusters':True, \
        'enable_snippets':True, \
        'search_field':'name', \
        'showClusters':True, \
        'page': ''
}
    
    header = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
    AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'}

    main_link = 'https://hh.ru/search/vacancy'
       
    response = requests.get(main_link, params=params, headers=header)
    
    if response.ok:
        soup = bs(response.text,'lxml')
        
        page_block = soup.find('div', {'data-qa': 'pager-block'})
        if not page_block:
            last_page = 1
        else:
            last_page = int(page_block.find_all('a', {'class': 'HH-Pager-Control'})[-2].getText())
    
    for page in range(0, last_page):
        params['page'] = page
        response = requests.get(main_link, params=params, headers=header)
        
        if response.ok:
            soup = bs(response.text,'lxml')
            
            vacancy_items = soup.find('div', {'data-qa': 'vacancy-serp__results'}) \
                                        .find_all('div', {'class': 'vacancy-serp-item'})
                
            for item in vacancy_items:
                vacancy_data.append(_parser_item_hh(item))
                
    return vacancy_data

In [3]:
def _parser_item_hh(item):

    vacancy_data = {}
    
    # vacancy_name
    vacancy_name = item.find('span', {'class': 'resume-search-item__name'}) \
                        .getText() \
                        .replace(u'\xa0', u' ')
    
    vacancy_data['vacancy_name'] = vacancy_name
    
    # company_name
    company_name = item.find('div', {'class': 'vacancy-serp-item__meta-info'}) \
                        .find('a') \
                        .getText()
    
    vacancy_data['company_name'] = company_name
    
    # city
    city = item.find('span', {'class': 'vacancy-serp-item__meta-info'}) \
                .getText() \
                .split(', ')[0]
    
    vacancy_data['city'] = city
      
    #salary
    salary = item.find('span', {'data-qa': 'vacancy-serp__vacancy-compensation'})
    if not salary:
        salary_min = None
        salary_max = None
        salary_currency = None
    else:
        salary = salary.getText() \
                        .replace(u'\xa0', u'')
        
        salary = re.split(r'\s|-', salary)
        
        if salary[0] == 'до':
            salary_min = None
            salary_max = int(salary[1])
        elif salary[0] == 'от':
            salary_min = int(salary[1])
            salary_max = None
        else:
            salary_min = int(salary[0])
            salary_max = int(salary[1])            
        
        salary_currency = salary[2]
        
    vacancy_data['salary_min'] = salary_min
    vacancy_data['salary_max'] = salary_max
    vacancy_data['salary_currency'] = salary_currency
    
    # link
    is_ad = item.find('a', {'data-qa': 'vacancy-serp__vacancy_response'}) \
                .getText()
    
    vacancy_link = item.find('span', {'class': 'resume-search-item__name'}) \
                        .find('a')['href']
    
    if is_ad != 'Реклама':
        vacancy_link = vacancy_link.split('?')[0]
    
    vacancy_data['vacancy_link'] = vacancy_link 
    
    # site
    vacancy_data['site'] = 'hh.ru'
    
    return vacancy_data

In [4]:
def _parser_superjob(vacancy):

    vacancy_data = []
    
    params = {
        'keywords': vacancy, \
        'profession_only': '1', \
        'page': ''
    }
    
    header = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
    AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'}

    main_link = 'https://russia.superjob.ru/vacancy/search/'
       
    response = requests.get(main_link, params=params, headers=header)
    
    if response.ok:
        soup = bs(response.text,'lxml')
        
        page_block = soup.find('a', {'class': 'f-test-button-1'})
        if not page_block:
            last_page = 1
        else:
            page_block = page_block.findParent()
            last_page = int(page_block.find_all('a')[-2].getText())
    
    for page in range(0, last_page + 1):
        params['page'] = page
        response = requests.get(main_link, params=params, headers=header)
        
        if response.ok:
            soup = bs(response.text,'lxml')
            
            vacancy_items = soup.find_all('div', {'class': 'f-test-vacancy-item'})
                
            for item in vacancy_items:
                vacancy_data.append(_parser_item_superjob(item))
                
    return vacancy_data

In [5]:
def _parser_item_superjob(item):

    vacancy_data = {}
    
    # vacancy_name
    vacancy_name = item.find_all('a')

    vacancy_name = vacancy_name[0].getText()
    vacancy_data['vacancy_name'] = vacancy_name
    
    # company_name
    company_name = item.find('span', {'class': 'f-test-text-vacancy-item-company-name'})
    
    if not company_name:
        company_name = None
    else:
        company_name = company_name.getText()
    
    vacancy_data['company_name'] = company_name
    
    # city
    company_location = item.find('span', {'class': 'f-test-text-company-item-location'}) \
                            .findChildren()[2] \
                            .getText() \
                            .split(',')
    
    vacancy_data['city'] = company_location[0]
      
    #salary
    salary = item.find('span', {'class': 'f-test-text-company-item-salary'}) \
               .findChildren()
    
    if not salary or len(salary) == 1 or len(salary) == 5:
        salary_min = None
        salary_max = None
        salary_currency = None
    else:
        is_check_salary = item.find('span', {'class': 'f-test-text-company-item-salary'}) \
                        .getText() \
                           .replace(u'\xa0', u' ') \
                         .split(' ', 1)[0]
        salary = salary[0].getText().replace(u'\xa0', u' ')
        salary_string = re.sub('(?<=\d)\s(?=\d)', '', salary)
        salary_string = salary_string.lower()
        words = re.findall(r'\w+', salary_string)
        
        if is_check_salary == 'до'or len(salary) == 2:
            salary_min = None
            salary_max = words[1]
            salary_currency = salary.split()[-1]
        elif is_check_salary == 'от':
            salary_min = words[1]
            salary_max = None
            salary_currency = salary.split()[-1]
        elif is_check_salary == 'По':
            pass
        else:
            salary_min = words[0]
            salary_max = words[1]
            salary_currency = salary.split()[-1]            
    
    vacancy_data['salary_min'] = salary_min
    vacancy_data['salary_max'] = salary_max
    vacancy_data['salary_currency'] = salary_currency
    
    # link
    vacancy_link = item.find('a', {'class': 'icMQ_'})
    if vacancy_link:
        vacancy_link = vacancy_link['href']
        vacancy_data['vacancy_link'] = f'https://russia.superjob.ru{vacancy_link}'
    
    # site
        vacancy_data['site'] = 'superjob.ru'
    
    return vacancy_data

In [6]:
def parser_vacancy(vacancy):
        
    vacancy_data = []
    vacancy_data.extend(_parser_hh(vacancy))
    vacancy_data.extend(_parser_superjob(vacancy))

    df = pd.DataFrame(vacancy_data)

    return df

In [7]:
vacancy = 'Data Scientist'
df = parser_vacancy(vacancy)

In [8]:
df.tail(30)

,vacancy_name,company_name,city,salary_min,salary_max,salary_currency,vacancy_link,site
228,Data scientist (NLP),"ПАО МегаФон, IT",Москва,NaN,NaN,None,https://hh.ru/vacancy/37766560,hh.ru
229,Middle / Senior Data Scientist,Сбербанк для экспертов,Москва,NaN,NaN,None,https://hh.ru/vacancy/36313639,hh.ru
230,Data Scientist,ООО IHS Global,Минск,NaN,NaN,None,https://hh.ru/vacancy/37645302,hh.ru
231,Data Scientist/Senior Data Scientist,СИТИЛИНК,Москва,NaN,NaN,None,https://hh.ru/vacancy/37731417,hh.ru
232,Data Scientist / Engineer (Machine Learning / ...,Sensoleak LTD,Москва,NaN,NaN,None,https://hh.ru/vacancy/37662632,hh.ru
233,Data Scientist (NLP),Сбербанк для экспертов,Москва,NaN,NaN,None,https://hh.ru/vacancy/37183492,hh.ru
234,Аналитик данных / Data scientist,"Программный Продукт, ИТ-компания",Москва,NaN,NaN,None,https://hh.ru/vacancy/36953584,hh.ru
235,Senior Data Scientist,ООО Flo,Минск,NaN,NaN,None,https://hh.ru/vacancy/37764024,hh.ru
236,Lead / Senior Data Scientist (Predictive analy...,"ОТП Банк, АО (OTP bank)",Москва,NaN,NaN,None,https://hh.ru/vacancy/37105028,hh.ru
237,Главный специалист Управления анализа и модели...,Московский Кредитный Банк,Москва,NaN,NaN,None,https://hh.ru/vacancy/37668113,hh.ru
